In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

plt.rcParams['font.sans-serif'] = ['Arial']
plt.rcParams['font.size'] = 11

from matplotlib_inline import backend_inline
backend_inline.set_matplotlib_formats('svg')

# layout of plotly graph objects
mlayout = {'title':'TITLE', 'width':800, 'height':300, 'margin':dict(l=10,r=10,t=50,b=10)}

# Read trade history file

Read the trading history csv file downloaded from Rakuten Securities. The code expects strict formatting of the spreadsheet.

Modifications may be needed for history files from other sources.

In [2]:
fname = 'tradehistory(INVST)_20220731.csv'

# the csv file follows a strict format and uses Shift JIS encoding
df_trades = pd.read_csv(fname, parse_dates=True, thousands=',', encoding='Shift JIS')

# drop unecessary columns from dataframe and replace Japanese headers with English ones cuz why not
df_trades.drop(['受渡日','分配金','口座','取引','買付方法','経費','為替レート','受付金額[現地通貨]'], axis=1, inplace=True)
df_trades.rename(columns={'約定日':'date', 
                          'ファンド名':'name',	
                          '数量［口］':'quantity', 
                          '単価':'price',	
                          '受渡金額/(ポイント利用)[円]':'cost/(point)',
                          '決済通貨':'currency'}, 
                          inplace=True)

df_trades['date'] = pd.to_datetime(df_trades['date'])

# replace asset names with simpler handles 
replace_titles = {'eMAXIS Slim 国内株式(TOPIX)':'TOPIX',
                  'eMAXIS Slim 国内リートインデックス':'Domestic Reit',
                  'eMAXIS Slim 先進国債券インデックス':'Advanced government bond',
                  'eMAXIS Slim 全世界株式(除く日本)':'Global equities (excluding Japan)',
                  'eMAXIS Slim 米国株式(S&P500)':'S&P500',
                  'eMAXIS NASDAQ100インデックス':'NASDAQ100'}

df_trades.replace(replace_titles,inplace=True)
df_trades.head()

,date,name,quantity,price,cost/(point),currency
0,2021-08-02,TOPIX,7259,13777,"10,000",円
1,2021-08-02,Domestic Reit,9799,10205,"10,000",円
2,2021-08-03,Advanced government bond,8459,11822,"10,000",円
3,2021-08-03,Global equities (excluding Japan),6454,15495,"10,000",円
4,2021-08-03,S&P500,6013,16632,"10,000",円


List of asset names which will be referenced later

In [7]:
assets = df_trades['name'].unique()
assets

array(['TOPIX', 'Domestic Reit', 'Advanced government bond',
       'Global equities (excluding Japan)', 'S&P500', 'NASDAQ100'],
      dtype=object)

# Download price histories

Download historical price data from an online source. Here, I used emaxis.jp. Modifications may be necessary for other sources.

You will need to curate your own url list for your instruments in the trade history.

In [26]:
#! Warning! Running this cell will overwrite existing historical files.

import urllib.request

# download price histories and save csv files to /historical_data
# will take a few second depending on the number of items in url_list.

# keys must match the asset names in 'assets'
url_list = {'TOPIX':
            'https://emaxis.jp/content/csv/fundCsv.php?fund_cd=252634',

            'Domestic Reit':
            'https://emaxis.jp/content/csv/fundCsv.php?fund_cd=253669',

            'Advanced government bond':
            'https://emaxis.jp/content/csv/fundCsv.php?fund_cd=252667',

            'Global equities (excluding Japan)':
            'https://emaxis.jp/content/csv/fundCsv.php?fund_cd=253209',

            'S&P500':
            'https://emaxis.jp/content/csv/fundCsv.php?fund_cd=253266',

            'NASDAQ100':
            'https://emaxis.jp/content/csv/fundCsv.php?fund_cd=254062'}


for key in url_list:
    url = url_list[key]
    title = "historical_data/{0}.csv".format(key)
    urllib.request.urlretrieve(url,title)

# check consistency of url_list and assets (all assets must be defined in url_list)
if (not set(assets).issubset(set(url_list.keys()))):    
    raise Exception('Asset(s) missing in the url_list. Check the asset names and url_list.')

In [31]:
# individual price history files are combined into one dataframe 
df_all = pd.DataFrame()

for asset in assets:
    asset_file = "historical_data/{0}.csv".format(asset)
    df_hist = pd.read_csv(asset_file, skiprows = [0,1], names = ['date','nav_void', 'nav','div', 'aum'],
                    parse_dates = True, index_col = 0, encoding='Shift JIS')

    df_hist[asset] = df_hist['nav']
    df_hist = df_hist.loc[:,[asset, 'div']]

    df_all[asset] = df_hist[asset]

df_all.to_csv('df_all.csv')

In [29]:
pd.options.plotting.backend = "plotly"

# trim the data from a start date
df_all = df_all['2019-12-29':]

# plot relative price increase from a start date (normalization)
df_cum = (df_all / df_all.iloc[0,:])

df_cum.plot()

# Average acquisition price

In [32]:
# this df_hist is structure is different from the above one
df_hist = pd.read_csv('df_all.csv', parse_dates=True)
df_hist.index = pd.to_datetime(df_hist['date'])

# trim the history from start date
df_hist = df_hist[df_hist['date'] > '2021-01-01']     

avg_acq = dict(pd.DataFrame())


for asset in assets:
    # get instrument and create a local copy
    instm = df_trades[df_trades['name']==asset].copy()

    # calculate average acquisition price
    instm['avg_acq'] = np.zeros(len(instm))
    for i, row in instm.iterrows():        
        instm.loc[i,'avg_acq'] = sum(instm.loc[:i+1,'price']*instm.loc[:i+1,'quantity'])/sum(instm.loc[:i+1,'quantity'])

    avg_acq[asset] = instm.loc[:,['date', 'avg_acq']]

    # plot chart for current asset
    mlayout['title'] = asset
    fig = go.Figure(layout=mlayout)

    fig.add_scatter(x=df_hist['date'], y=df_hist[asset], opacity=0.7, name='Price')
    fig.add_scatter(x=instm['date'], y=instm['price'], mode='lines+markers', name='Acquisition')
    fig.add_scatter(x=instm['date'], y=instm['avg_acq'], mode='lines+markers', name='Avg acqust.')

    fig.show()

# Profit & loss

Calculate profit history by substracting average acquisition price from the asset price on each date.

In [33]:
profit_hist = dict(pd.DataFrame())

# front fill average acquisition price timehistory
for asset in assets:
    # get instrument
    instm = df_trades[df_trades['name']==asset].copy()
 
    # combine date column with dates from price history
    combi = avg_acq[asset].combine_first(pd.DataFrame(df_hist['date']))
    combi.index = pd.to_datetime(combi['date'])
    combi.sort_index(inplace=True)
    combi.interpolate('ffill', inplace=True)
    
    profit_hist[asset] = pd.DataFrame(df_hist[asset] - combi['avg_acq']) 
    
    # plot chart to verify interpolated average acquisition price
    mlayout['title'] = asset
    fig = go.Figure(layout=mlayout)

    fig.add_scatter(x=df_hist['date'], y=df_hist[asset], opacity=0.7, name='Price')
    fig.add_scatter(x=avg_acq[asset]['date'], y=avg_acq[asset]['avg_acq'], name='Avg acq')
    fig.add_scatter(x=combi.index, y=combi['avg_acq'], opacity=0.7, name='Interp. avg acq')
    
    # fig.show()

In [34]:
# plot profit histories

for asset in assets:
    mlayout['title'] = asset
    fig = go.Figure(layout=mlayout)

    fig.add_scatter(x=profit_hist[asset].index, y=profit_hist[asset][0], name='Profit')

    print(asset, 'average %5.2f' %np.mean(profit_hist[asset][0]))
    fig.show()

TOPIX average 161.14


Domestic Reit average -220.14


Advanced government bond average 52.98


Global equities (excluding Japan) average 516.71


S&P500 average 1151.19


NASDAQ100 average -523.10
